In [1]:
import json
import pandas as pd
from glob import glob
from tqdm import tqdm
tqdm.pandas()
import numpy as np
from scripts.graph_funcs import *
from scripts.network_graph import make_network
import networkx as nx


In [2]:
file_list=glob('dataset/*/*.json')
df=pd.DataFrame.from_dict(json.load(open(file_list[0])), orient="index").T
for patent_loc in file_list[1:]:
    patent_file=json.load(open(patent_loc))
    temp_df=pd.DataFrame.from_dict(patent_file, orient="index").T
    df=pd.concat([df, temp_df], ignore_index=True)

In [3]:
df['publication_date']= pd.to_datetime(df['publication_date'])
df['publication_year']=df['publication_date'].apply(lambda k: k.year)

In [4]:
df.head()

,patent_id,title,assignee,publication_date,patent_link,inventors,classifications,abstract,claims,status,citations_own,citations_ftf,citedby_own,citedby_ftf,publication_year
0,US-10002325-B2,Knowledge representation systems and methods i...,Primal Fusion Inc.,2018-06-19,https://patents.google.com/patent/US10002325B2/en,"[Peter Joseph Sweeney, Ihab Francis Ilyas, Wu ...","[G06N5/02, G06F16/84, G06F17/30914, G06N7/005,...",Techniques for analyzing and synthesizing comp...,\n1. A method of modifying a computer-readable...,Active,"[US3943462A, US4532813A, US4972328A, US5056021...",[],"[US20150356418A1, US10248669B2, US10671577B2, ...","[US9378203B2, US7849090B2, US8849860B2, US9104...",2018
1,US-10162734-B1,Method and system for crowdsourcing software q...,Intuit Inc.,2018-12-25,https://patents.google.com/patent/US10162734B1/en,"[Igor A. Podgorny, Joseph Cessna, Christopher ...","[G06F11/366, G06N20/00, G06N20/20, G06N5/041, ...",Disclosed methods and systems crowdsource qual...,\n1. A computing system implemented method for...,Active,"[US5471382A, US5519608A, US6147975A, US2002011...",[],"[US20180032890A1, US10373064B2, US10394804B1, ...",[],2018
2,US-10210244-B1,Updating natural language interfaces by proces...,"Asapp, Inc.",2019-02-19,https://patents.google.com/patent/US10210244B1/en,"[Satchuthananthavale Rasiah Kuhan Branavan, Jo...","[G06F17/30654, G06F16/3329, G06F16/35, G06F17/...",A third-party company may assist companies in ...,\n1. A system for implementing natural languag...,Active,"[US5195034A, US5227971A, US5369575A, US6173279...","[US8374863B2, US7664644B1, US9558743B2, WO2015...","[US20190099224A1, US10387888B2, US10482875B2, ...","[US11593608B2, US11049023B1, US11393475B1, US2...",2019
3,US-10216983-B2,Techniques for assessing group level cognitive...,General Electric Company,2019-02-26,https://patents.google.com/patent/US10216983B2/en,"[Peter Henry Tu, Tao Gao, Jilin Tu]","[G06V10/62, G06K9/00335, G06N5/041, A61B5/0013...",A security monitoring technique includes recei...,"\n1. A method, comprising:\nreceiving data rel...",Active,"[US20040057556A1, US20060059113A1, US200601533...","[US7433493B1, US8195598B2, JP5366047B2, JP2011...","[US11381651B2, EP4004814A4]","[US10445565B2, US20180190377A1, JP6615800B2, U...",2019
4,US-10275453-B2,"Automatic, unsupervised paraphrase detection",International Business Machines Corporation,2019-04-30,https://patents.google.com/patent/US10275453B2/en,"[Laura J. BENNETT, Lakshminarayanan Krishnamur...","[G06F17/2775, G06N5/041, G06F16/243, G06F17/27...",A method is disclosed for identifying paraphra...,\n1. A computer-implementable method for ident...,Active,"[US6161084A, US7392185B2, US8271453B1, CA24844...",[],[US10740401B2],"[CN111555954B, US10083451B2, US10275452B2, US1...",2019


In [26]:
G2=make_network(df, use_citations=True, use_abstract=True, file_loc='dataset/network.adjlist')

100%|██| 271216/271216 [10:11<00:00, 443.19it/s]


In [19]:
G=make_network(df, use_abstract=True, use_cpc=True, file_loc='dataset/network2.adjlist')

100%|██| 271216/271216 [08:49<00:00, 512.32it/s]


In [5]:
G=nx.read_adjlist('dataset/network.adjlist')

In [6]:
G2=nx.read_adjlist('dataset/network2.adjlist')

In [7]:
citation_patent={}

for idx in df.index:
    for citation in df['citations_own'][idx]:
        if citation in citation_patent:
            citation_patent[citation].append(df['patent_id'][idx])
        else:
            citation_patent[citation] = [df['citations_own'][idx]]

In [8]:
patent_to_check='US-4658370-A'

In [15]:
starter_nodes=[]
for citation in df[df['patent_id']==patent_to_check]['citations_own'].values[0]:
    starter_nodes.append(np.random.choice(citation_patent[citation][0]))

In [22]:
classificiation_patent={}

for idx in df.index:
    for cpc in df['classifications'][idx]:
        if cpc in classificiation_patent:
            classificiation_patent[cpc].append(df['patent_id'][idx])
        else:
            classificiation_patent[cpc]=[df['patent_id'][idx]]

In [23]:
patent_to_check='US-4658370-A'

In [24]:
starter_nodes=[]
for cpc in df[df['patent_id']==patent_to_check]['classifications'].values[0]:
    starter_nodes.append(np.random.choice(classificiation_patent[cpc]))

In [8]:
random_walks=[]
for node in starter_nodes:
    random_walks.append(random_walk(G, node))

In [9]:
from functools import reduce

common_patents = list(reduce(lambda i, j: i & j, (set(x) for x in random_walks)))

In [10]:
matched_nodes={}
if patent_to_check in common_patents:
    common_patents.remove(patent_to_check)
for patent in common_patents:
    matched_nodes[patent]=get_graph_similarity(patent_to_check, patent)

In [11]:
matched_nodes

{'US-11132609-B2': [['1', '21'],
  ['58', '21'],
  ['66', '1'],
  ['66', '21'],
  ['61', '1'],
  ['61', '21'],
  ['61', '22']],
 'US-11022719-B2': [['66', '2'],
  ['66', '16'],
  ['61', '15'],
  ['61', '2'],
  ['61', '16']],
 'US-11475330-B2': [['59', '1'], ['66', '1'], ['61', '1']],
 'US-11423886-B2': [['59', '19']],
 'US-11403526-B2': [],
 'US-11357502-B2': [],
 'US-11501194-B2': [['50', '1'], ['59', '11'], ['66', '11']],
 'US-11675943-B2': [['58', '1'], ['59', '1'], ['66', '1'], ['61', '1']],
 'US-9349100-B2': [],
 'US-2021232955-A1': [['59', '1'],
  ['59', '7'],
  ['59', '13'],
  ['59', '19'],
  ['59', '22'],
  ['66', '1'],
  ['66', '7'],
  ['66', '13'],
  ['66', '19'],
  ['66', '22'],
  ['60', '1'],
  ['60', '7'],
  ['60', '13'],
  ['61', '1'],
  ['61', '7'],
  ['61', '13'],
  ['61', '19'],
  ['61', '22'],
  ['63', '1'],
  ['63', '13']]}